In [ ]:
#install.packages("R.matlab")
#install.packages("ggplot2")
#install.packages("gridextra")
#install.packages("plotly")
#devtools::install_github('GRousselet/rogme')
#install.packages("tidyverse")

In [ ]:
library(rogme)
library(tibble)
library(ggplot2)
library(R.matlab)
library(plotly)
library(htmlwidgets)
library(tidyverse)
#library(cowplot)
set.seed(123)

# Percent difference function

In [ ]:
percentage_difference <- function(value, value_two) {
  pctd = abs(value - value_two) / ((value + value_two) / 2) * 100
  return(pctd)
} 

# I/O

In [ ]:
subs = list("sub-invivo1","sub-invivo2","sub-invivo3")
metrics = list("T1","MTR","MTsat")
sessions <- list("ses-rth750rev","ses-rthPRIrev","ses-rthSKYrev","ses-vendor750rev","ses-vendorPRIrev","ses-vendorSKYrev")

## T1, MTR and MTSat individual D_PB_HD_SF comparisons

Percentile bootstrapped comparison of dependent samples using Harrell-Davis quantile estimator.

In [ ]:
# Please change derivatives directory below if running locally.
metrics = list("MTsat")
for (metric in metrics){
for (sub in subs) {

derivativesDir <- paste("/Users/agah/Desktop/KuzuData/ds-toronto/derivatives/qMRFlow/",sub ,"/",sep = "")

T1DIR = paste(derivativesDir,"T1_SF_PLOTS",sep="")
MTRDIR = paste(derivativesDir,"MTR_SF_PLOTS",sep="")
MTSDIR = paste(derivativesDir,"MTS_SF_PLOTS",sep="")
dir.create(T1DIR)
dir.create(MTRDIR)
dir.create(MTSDIR)

sessions <- list("ses-rth750rev","ses-rthPRIrev","ses-rthSKYrev","ses-vendor750rev","ses-vendorPRIrev","ses-vendorSKYrev")
# Create dataframe with 37k samples per session
nSamples = 37000
df <- tibble(N = 1:nSamples)
for (i in seq(from=1, to=length(sessions), by=1)) {
  curMat <- readMat(paste(derivativesDir,sessions[i],"/stat/",sub,"_",sessions[i],"_desc-wm_metrics.mat",sep = ""))
  #curTest <- readMat(paste(derivativesDir,sessions[i+1],"/stat/sub-invivo1_",sessions[i+1],"_desc-wm_metrics.mat",sep = ""))
  curMetric <- curMat[[metric]]
  #tmp <- Map(function(x,y) mean(c(x,y)), curRetest[[metric]], curRetest[[metric]])
  #curAvg <- as.numeric(unlist(tmp))
  if (metric == "MTR"){
      OUTDIR = MTRDIR
      PLTRANGE = 10.1 
      curMetric <- curMetric[curMetric > 35]
      curMetric <- curMetric[curMetric < 70]
  }else if (metric == "MTsat"){
      OUTDIR = MTSDIR
      PLTRANGE = 1.7
      curMetric <- curMetric[curMetric > 1]
      curMetric <- curMetric[curMetric < 8]
  }else if (metric == "T1"){
      OUTDIR = T1DIR
      PLTRANGE = 0.5
      curMetric <- curMetric[curMetric > 0]
      curMetric <- curMetric[curMetric < 3]    
  }
  # Do not choose the same element more than once
  print(length(curMetric))
  curMetric <- sample (curMetric, size=nSamples, replace =F)          
  varName <- toString(sessions[i])
  varName <- substr(varName,5,nchar(varName)-3)
  print(varName)
  df <- mutate(df, !!varName := curMetric)       
}
             
neutral = list("rth750","rthPRI","rthSKY")
native  = list("vendor750","vendorPRI","vendorSKY")
combns <- combn(c(1,2,3),2)

for (j in seq(from=1,to=2, by=1)){
for (i in seq(from=1,to=3, by=1)){

if (j==1){
    sel1 = toString(neutral[combns[,i][1]])
    sel2 = toString(neutral[combns[,i][2]])
    }
else
{
    sel1 = toString(native[combns[,i][1]])
    sel2 = toString(native[combns[,i][2]])
}

paste(sel1,"vs",sel2," T1",' in progress',sep="")
    
grp1 <- df[[sel1]]
grp2 <- df[[sel2]]

sfInput <- mkt2(grp1,grp2,group_labels=c(sel1,sel2))
sf <- shiftdhd_pbci(
  data = sfInput,
  formula = obs ~ gr,
  nboot = 250,
  alpha = 0.05/30
)

sf[[1]]['pctdif'] = percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])
    
write.csv(sf,paste(OUTDIR,"/",sub,"_",sel1,"vs",sel2,"_",metric,"_rev",'.csv',sep=""))
print(min(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])))
print(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])[5])
print(max(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])))

    
th <- ggplot2::theme_dark() + ggplot2::theme(
  panel.border = element_blank(),
  panel.background = element_rect(fill = "black"),
  plot.background = element_rect(fill = "black"),
  legend.background = element_rect(fill="white",size=0.5),  
  legend.text=element_text(color="white",size=12),    
  axis.line=element_blank(),
  axis.text = element_blank(),
  axis.title = element_blank(),
  axis.text.x = element_text(colour="white",size=18,face='bold'),
  axis.text.y = element_text(colour="white",size=18,face='bold')
  ) 

if (j==1){
    lnclr = "darkgray"
    flclr = "red"
} else
{
    lnclr = "darkgray"
    flclr = "blue"
}
    
psf <- plot_sf(
  data = sf,
  plot_theme = 1,
  symb_col = "black",
  symb_fill = flclr,
  symb_size = 8,
  symb_shape = 21,
  diffint_col = "white",
  diffint_size = 1.5,
  q_line_col = "black",
  q_line_alpha = 0.8,
  q_line_size = 1.8,
  theme2_alpha = NULL
) 


curPlot <- psf[[1]] + geom_hline(yintercept = 0,colour=lnclr, linetype = "longdash",size=1) + ylim(-PLTRANGE,PLTRANGE)
ggsave(
  filename = paste(OUTDIR,"/",sub,"_",sel1,"vs",sel2,"_",metric,"_rev",'.png',sep=""),
  plot = curPlot,
  device = NULL,
  path = NULL,
  scale = 1,
  width = NA,
  height = NA,
  units = c("mm"),
  dpi = 300,
  limitsize = TRUE
)
}}
}}
#+  scale_x_continuous(breaks = seq(6, 20, 2), limits = c(8, 20)) +  scale_y_continuous(breaks = seq(-8, 4, 2), limits = c(-8, 4))

# Define helper functions

In [ ]:
obj2plotly <- function(curObj,acq,plim){

if (acq=="rth"){
color="#d62728"    
}else{
color="#1f77b4"
}
    
p<-plot_hsf_pb(curObj, viridis_option="D", ind_line_alpha=0.9, ind_line_size=1, gp_line_colour = "red", gp_point_colour=color,gp_point_size=4,gp_line_size=1.5) 
#names(p$data)[names(p$data) == "participant"] <- "participant"
p$labels$colour = ""
p$data$participants = revalue(p$data$participants,c("1"="sub1", "2"="sub2","3"="sub3"))

th <- ggplot2::theme_bw() + ggplot2::theme(
  # get rid of panel grids
  #panel.grid.major = element_blank(),
  #panel.grid.minor = element_blank(),
  legend.title = element_blank(),  
  # Change plot and panel background
  #panel.background = element_rect(fill = "rgba(45, 59, 70,1)"),
  #plot.background = element_rect(fill = "black"),
  #legend.background = element_rect(fill="rgba(45, 59, 70,1)",size=0.5),  
  #legend.text=element_text(color="white",size=12),  
  #panel.border=element_blank(),   
  axis.line=element_blank(),
    legend.position = "none",
  axis.ticks = element_blank(),
  #axis.text = element_blank(),
  axis.title = element_blank(),
  panel.border = element_rect(colour = "black", fill=NA, size=0.5)
  ) 



k <-p  + th + aes(x = dec, y = diff, colour = participants) + ggtitle("")

k$labels$colour = revalue(k$labels$colour,c("red"="group"))

axx <- list(
  zeroline = FALSE,
  tickmode = "array",
  tickvals = c("0.1","0.2","0.3","0.4","0.5","0.6","0.7","0.8","0.9"),
  ticktext = c("q1","q2","q3","q4","median","q6","q7","q8","q9"),
  showticklabels= TRUE,
  ticksuffix=" "
)
    
axy <- list(
  zeroline = FALSE,   
  range = c(-plim,plim),
  overlaying = "y2",
  showticklabels= FALSE
  # The range is fixed per panel (3 MAD from both side) to infer the relative effect size   
)

axy2 <- list(
  range = c(-plim,plim),
    showticklabels= TRUE,
    ticksuffix=" "
  # The range is fixed per panel (3 MAD from both side) to infer the relative effect size   
) 
    
# Create a Purple --> Orange --> Green divergent colorscale for segment identificaiton. 
# The convention will be followed throughout the reports.     
#style(traces=1,opacity=0.8,line = list(shape = "spline",color="#762a83",width=3)) %>%    
figObj <- ggplotly(k) %>% 
            layout(height=800,width=800,xaxis=axx,yaxis = axy, yaxis2 = axy2, margin=c(5,5,5,5)) %>% # SWITCH BACK TO 400 x 400
            style(traces=1,mode="lines+markers",opacity=1,marker=list(size=10),line = list(shape = "spline",color="purple",width=5)) %>%
            style(traces=2,mode="lines+markers",opacity=1,marker=list(size=10),line = list(shape = "spline",color="hotpink",width=5)) %>% 
            style(traces=3,mode="lines+markers",opacity=1,marker=list(size=10),line = list(shape = "spline",color="mediumvioletred",width=5)) %>% 
            style(traces=4,opacity=1,line = list(color="black",width=2,dash="dash"),yaxis = 'y2') %>% # zero line
            style(traces=5,mode = "lines+markers",opacity=0.9,line = list(shape = "spline",width=3,color="black"),marker = list(color=color,size=30,line=list(color="black",width=1))) %>% # group markers
            style(traces=6,opacity=0,line = list(shape = "spline",color="#762a83",width=3))
return(figObj)    
} #END FUNCTION DEF

In [ ]:
get_vector <- function(subject, session, metric, nSamples){
  derivativesDir <- paste("/Users/agah/Desktop/KuzuData/ds-toronto/derivatives/qMRFlow/",subject ,"/",sep = "")  
  curMat <- readMat(paste(derivativesDir,session,"/stat/",subject,"_",session,"_desc-wm_metrics.mat",sep = ""))
  curMetric <- curMat[[metric]]
  if (metric == "MTR"){
      curMetric <- curMetric[!sapply(curMetric, is.nan)]
      curMetric <- curMetric[curMetric > 35]
      curMetric <- curMetric[curMetric < 70]
  }else if (metric == "MTsat"){
      curMetric <- curMetric[!sapply(curMetric, is.nan)]
      curMetric <- curMetric[curMetric > 1]
      curMetric <- curMetric[curMetric < 8]
  }else if (metric == "T1"){
      curMetric <- curMetric[!sapply(curMetric, is.nan)]
      curMetric <- curMetric[curMetric > 0]
      curMetric <- curMetric[curMetric < 3]    
  } 
  curMetric <- sample (curMetric, size=nSamples, replace =F)
  return(curMetric)  
}

get_df <- function(ses1,ses2,metric,nSamples){
df2 <- tibble(participant=as_factor(c(rep(1,nSamples*2),rep(2,nSamples*2),rep(3,nSamples*2))))
df2["condition"] = as_factor(rep(c(rep(ses1,nSamples),rep(ses2,nSamples)),3))
curComp = c(get_vector("sub-invivo1",ses1,metric,nSamples),
       get_vector("sub-invivo1",ses2,metric,nSamples),
       get_vector("sub-invivo2",ses1,metric,nSamples),
       get_vector("sub-invivo2",ses2,metric,nSamples),
       get_vector("sub-invivo3",ses1,metric,nSamples),
       get_vector("sub-invivo3",ses2,metric,nSamples))
df2["metric"] = curComp
return(df2)
}

# Perform HSF analysis

In [ ]:
derivativesDir <- paste("/Users/agah/Desktop/KuzuData/ds-toronto/derivatives/qMRFlow/",sep = "")

HSFDIR = paste(derivativesDir,"HSF/",sep="")
dir.create(HSFDIR)

sessions <- list("ses-rth750rev","ses-rthPRIrev","ses-rthSKYrev","ses-vendor750rev","ses-vendorPRIrev","ses-vendorSKYrev")
nSamples = 37000

neutral = list("ses-rth750rev","ses-rthPRIrev","ses-rthSKYrev")
native  = list("ses-vendor750rev","ses-vendorPRIrev","ses-vendorSKYrev")
combns <- combn(c(1,2,3),2)

metrics = list("T1","MTR","MTsat")
for (metric in metrics){

if (metric == "MTR"){
      PLTRANGE = 10.3 
  }else if (metric == "MTsat"){
      PLTRANGE = 1.8
  }else if (metric == "T1"){
      PLTRANGE = 0.6
  }
for (j in seq(from=1,to=2, by=1)){
for (i in seq(from=1,to=3, by=1)){

if (j==1){
    sel1 = toString(neutral[combns[,i][1]])
    sel2 = toString(neutral[combns[,i][2]])
    clr = "rth"
    }
else
{
    clr = "vendor"
    sel1 = toString(native[combns[,i][1]])
    sel2 = toString(native[combns[,i][2]])
}

print(paste(sel1,"vs",sel2," T1",' in progress',sep=""))
    

cur_df <-get_df(sel1,sel2,metric,nSamples)
out <- hsf_pb(
  data = cur_df,
  formula = metric ~ condition + participant,
  qseq = seq(0.1, 0.9, 0.1),
  tr = 0,
  alpha = 0.05,
  qtype = 8,
  todo = c(1, 2),
  nboot = 750)
# Orca does not recognize file paths, no need to use sys calls, outputs will be dumped in the working dir.    
orca(obj2plotly(out,clr,PLTRANGE), file = paste(substr(sel1,5,nchar(sel1)-3),"vs",substr(sel2,5,nchar(sel2)-3),"_HSF_",metric,".png",sep=""))    
}}}

# Plot bootstrapped differences at each decile for the hierachical shift function analyis

In [ ]:
derivativesDir <- paste("/Users/agah/Desktop/KuzuData/ds-toronto/derivatives/qMRFlow/",sep = "")

HSFDIR = paste(derivativesDir,"HSF/",sep="")
dir.create(HSFDIR)

sessions <- list("ses-rth750rev","ses-rthPRIrev","ses-rthSKYrev","ses-vendor750rev","ses-vendorPRIrev","ses-vendorSKYrev")
nSamples = 37000

neutral = list("ses-rth750rev","ses-rthPRIrev","ses-rthSKYrev")
native  = list("ses-vendor750rev","ses-vendorPRIrev","ses-vendorSKYrev")
combns <- combn(c(1,2,3),2)

metrics = list("T1","MTR","MTsat")
for (metric in metrics){

if (metric == "MTR"){
      PLTRANGE = 10.3 
  }else if (metric == "MTsat"){
      PLTRANGE = 1.8
  }else if (metric == "T1"){
      PLTRANGE = 0.6
  }
for (j in seq(from=1,to=2, by=1)){
for (i in seq(from=1,to=3, by=1)){

if (j==1){
    sel1 = toString(neutral[combns[,i][1]])
    sel2 = toString(neutral[combns[,i][2]])
    clr = "#d62728"
    }
else
{
    clr = "#1f77b4"
    sel1 = toString(native[combns[,i][1]])
    sel2 = toString(native[combns[,i][2]])
}

print(paste(sel1,"vs",sel2," T1",' in progress',sep=""))
    

cur_df <-get_df(sel1,sel2,metric,nSamples)
out <- hsf_pb(
  data = cur_df,
  formula = metric ~ condition + participant,
  todo = c(1, 2),
  nboot = 750)
# Orca does not recognize file paths, no need to use sys calls, outputs will be dumped in the working dir.
obj <- plot_hsf_pb_dist(out,fill_colour=clr,point_interv="median")
ggsave(
  filename = paste(substr(sel1,5,nchar(sel1)-3),"vs",substr(sel2,5,nchar(sel2)-3),"_HSFdif_",metric,".png",sep=""),
  plot = obj,
  device = NULL,
  path = NULL,
  scale = 1,
  width = NA,
  height = NA,
  units = c("mm"),
  dpi = 300,
  limitsize = TRUE
)
#orca(obj, file = paste(substr(sel1,5,nchar(sel1)-3),"vs",substr(sel2,5,nchar(sel2)-3),"_HSFdif_",metric,".png",sep=""))    
}}}